In [1]:
import json

import evaluate
import torch as tt
import pandas as pd
import numpy as np

from typing import Any, Dict, Union

from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedModel, PreTrainedTokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EvalPrediction

In [2]:
# models:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model_best = T5ForConditionalGeneration.from_pretrained("Ru-RACE-T5-title-best").to(tt.device("cuda:0"))
model_last = T5ForConditionalGeneration.from_pretrained("RuT5-RACE-title/checkpoint-87500").to(tt.device("cuda:0"))

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
option_id_dict = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3
}

def to_new_format(example: dict[str, Union[str, list[str]]]) -> str:
  inp, label = '', ''
  example["options_ru"] = [option for option in example["options_ru"] if option]
  right_answer = example['options_ru'][option_id_dict[example['answer']]]
  inp += example['article_ru'] + "\n" + "ВОПРОС: Какое название лучше всего подойдёт для этого текста? "
  inp += f"ПРАВИЛЬНЫЙ ОТВЕТ: {right_answer}"
  inp += "\nНЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:\n"
  for option in example["options_ru"]:
      if option != right_answer:
          label += f"\n  {option}"
  return {"inp": inp, "distractors": label, "right_answer": right_answer}

In [4]:
with open("title_dataset_pretty_filtered.json", 'r', encoding="utf8") as inp:
    title_dataset = json.load(inp)

title_dataset_train, title_dataset_val, title_dataset_test = title_dataset["train"], title_dataset["val"], title_dataset["test"]
title_dataset_train = Dataset.from_list(title_dataset_train)
title_dataset_val = Dataset.from_list(title_dataset_val)
title_dataset_test = Dataset.from_list(title_dataset_test)

title_dataset_train = title_dataset_train.map(to_new_format)
title_dataset_val = title_dataset_val.map(to_new_format)
title_dataset_test = title_dataset_test.map(to_new_format)

Map:   0%|          | 0/4375 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Evaluate on the whole dataset:

In [5]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inp"]
    )
    labels = tokenizer(
        examples["distractors"]
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


title_dataset_train = title_dataset_train.map(preprocess_function, batched=True, batch_size=2)
title_dataset_val = title_dataset_val.map(preprocess_function, batched=True, batch_size=2)
title_dataset_test = title_dataset_test.map(preprocess_function, batched=True, batch_size=2)


BATCH_SIZE  = 1
NUM_TRAIN_EPOCHS = 20
MODEL_NAME="RuT5-RACE-title-eval"

args = Seq2SeqTrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy="epoch", save_strategy="epoch",
    # evaluation_strategy="steps", eval_steps=100, save_steps=100,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    prediction_loss_only=False,
    gradient_checkpointing=True,
    predict_with_generate=True, fp16=True,
    eval_accumulation_steps=1
)


def compute_metric_values(output: list[str], label_batch: list[str]) -> dict[str, Any]:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

def compute_metrics(eval_preds: EvalPrediction) -> dict[str, Any]:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels > 0, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print(len(preds), len(labels))

    preds = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in preds
    ]
    labels = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in labels
    ]

    metrics = compute_metric_values(preds, labels)
    metric_dict = {
        "bleu": metrics["bleu"]["bleu"],
        "sbleu": metrics["sbleu"]["score"],
        "rouge1": metrics["rouge"]["rouge1"],
        "rouge2": metrics["rouge"]["rouge2"],
        "rougeL": metrics["rouge"]["rougeL"],
        "rougeLsum": metrics["rouge"]["rougeLsum"],
        "meteor": metrics["meteor"]["meteor"]
    }
    return metric_dict

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_last, padding=True)

trainer = Seq2SeqTrainer(
    model_last,
    args,
    train_dataset=title_dataset_train,
    eval_dataset=title_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Map:   0%|          | 0/4375 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [6]:
trainer.evaluate()

/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


219 219


{'eval_loss': 4.688899040222168,
 'eval_bleu': 0.07925729794697821,
 'eval_sbleu': 7.925729794697821,
 'eval_rouge1': 0.008480104370515329,
 'eval_rouge2': 0.0036529680365296807,
 'eval_rougeL': 0.008480104370515329,
 'eval_rougeLsum': 0.008480104370515329,
 'eval_meteor': 0.15350119191112346,
 'eval_runtime': 113.4037,
 'eval_samples_per_second': 1.931,
 'eval_steps_per_second': 1.931}

In [8]:
trainer.evaluate(title_dataset_test)

242 242


{'eval_loss': 4.64674186706543,
 'eval_bleu': 0.06429751741731445,
 'eval_sbleu': 6.429751741731442,
 'eval_rouge1': 0.012734785875281741,
 'eval_rouge2': 0.0013774104683195595,
 'eval_rougeL': 0.012734785875281741,
 'eval_rougeLsum': 0.012797395442023539,
 'eval_meteor': 0.14833763487535995,
 'eval_runtime': 143.9518,
 'eval_samples_per_second': 1.681,
 'eval_steps_per_second': 1.681}

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_best, padding=True)

trainer = Seq2SeqTrainer(
    model_best,
    args,
    train_dataset=title_dataset_train,
    eval_dataset=title_dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

219 219


{'eval_loss': 3.082942008972168,
 'eval_bleu': 0.01303553961201627,
 'eval_sbleu': 1.303553961201627,
 'eval_rouge1': 0.0,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0,
 'eval_rougeLsum': 0.0,
 'eval_meteor': 0.064937282371549,
 'eval_runtime': 103.8302,
 'eval_samples_per_second': 2.109,
 'eval_steps_per_second': 2.109}

In [10]:
trainer.evaluate(title_dataset_test)

242 242


{'eval_loss': 3.0236856937408447,
 'eval_bleu': 0.015322439643502146,
 'eval_sbleu': 1.532243964350215,
 'eval_rouge1': 0.004820936639118457,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.004132231404958678,
 'eval_rougeLsum': 0.004132231404958678,
 'eval_meteor': 0.08073514475478905,
 'eval_runtime': 119.6661,
 'eval_samples_per_second': 2.022,
 'eval_steps_per_second': 2.022}

In [5]:
def model_predict(text: str, model: PreTrainedModel) -> str:
    input_ = tokenizer([text], return_tensors="pt")
    output = model.generate(
        input_["input_ids"].to(tt.device("cuda:0"))
    )
    return tokenizer.batch_decode(output)[0]

In [6]:
print(title_dataset_test[42]["inp"])

В 70-е годы Организация Объединенных Наций организовала несколько важных совещаний по проблемам окружающей человека среды для изучения весьма серьезной проблемы. Мы, люди, уничтожаем мир вокруг нас. Мы должны научиться защищать их, иначе жизнь будет очень плохой для наших детей и внуков.
Эта проблема имеет несколько важных аспектов.
Население
Большинство проблем окружающей среды связано с ростом численности населения. В 1700 году в мире насчитывалось 635 миллионов человек; в 1900 году их было 1,6 миллиарда; в 1950 году - 2,5 миллиарда; и в 1980 году - 4,4 миллиарда. В 2010 году будет 7,3 миллиарда человек, больше людей нуждаются в воде, большем количестве продуктов питания, большем количестве древесины и большем количестве нефти.
Распределение
Ученые говорят, что в мире достаточно воды для всех, но в некоторых странах много воды, а в некоторых только немного. Некоторые районы получают весь дождь в течение одного сезона. Остальная часть года суха.
Нефтепродукты
Мы используем нефть в мир

In [7]:
print(title_dataset_test[42]["distractors"])


  Разрушенный мир
  Серьезная проблема, на которую нам следует обратить внимание
  Аспекты, которые разрушили наш мир


In [8]:
predict_model_best = model_predict(title_dataset_test[42]["inp"], model_best)
print(predict_model_best)

/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Как защитить окружающую среду Как защитить окружающую среду Как защитить окружающую среду</s>


In [9]:
predict_model_last = model_predict(title_dataset_test[42]["inp"], model_last)
print(predict_model_last)

<pad> Как защитить окружающую среду Глобальная проблема Как защитить окружающую среду</s>


In [10]:
print(title_dataset_test[100]["inp"])

Говорящие растения могут звучать как персонажи в сказке. Но недавние научные исследования показали, что растения общаются друг с другом и с другими живыми существами удивительно многими способами. Чтобы понять их, ученые говорят, нам просто нужно выучить их язык. Фермеры особенно заинтересованы в том, что говорят растения.
"Планты могут общаться со всеми видами организмов. Они могут общаться с гигантскими бактериями, другими растениями и насекомыми. Они делают это химически, - сказал Кахилл, профессор экологии Университета Альберты в Канаде.
Ученые-растители только начинают понимать этот химический "язык". Кейхил говорит, что исследования показали, например, что растения могут оценивать состояние окружающей их среды и принимать соответствующие меры.  Растения способны, например, сигнализировать боль или дискомфорт, вызываемые чем-либо от экстремальных температур до насекомого. Джек Шульц, профессор химической экологии Миссурийского университета, говорит, что, когда растение чувствует, 

In [11]:
print(title_dataset_test[100]["distractors"])


  Связь между растениями
  Химический "Language"
  Как растения защищают себя


In [12]:
predict_model_best = model_predict(title_dataset_test[100]["inp"], model_best)
print(predict_model_best)

<pad> Растения и люди Растения и люди Растения и люди</s>


In [13]:
predict_model_last = model_predict(title_dataset_test[100]["inp"], model_last)
print(predict_model_last)

<pad> Волшебный язык Растения могут общаться Растения могут говорить</s>


In [24]:
def get_metric_inputs_seq2seq(
    input_batch: list[str], #label_batch: list[str],
    model: PreTrainedModel, tokenizer: PreTrainedTokenizer
) -> list[str]:
    input_batch_ = tokenizer(input_batch, return_tensors="pt", padding=True)["input_ids"].to(tt.device("cuda:0"))
    # label_batch_ = tokenizer(label_batch, return_tensors="pt", padding=True)["input_ids"]

    # output_length = label_batch_.shape[-1]

    with tt.no_grad():
        output_batch = model.generate(input_batch_)

    output = [
        sent.replace("<pad>", " ").replace("</s>", " ").strip() for sent in tokenizer.batch_decode(output_batch)
    ]
    
    del input_batch_
    del output_batch
    # del label_batch_
    tt.cuda.empty_cache()

    return output

def compute_metrics(output: list[str], label_batch: list[str]) -> dict:
    metric_dict = {
        "bleu": bleu4.compute(predictions=output, references=[[label] for label in label_batch]),
        "sbleu": sbleu.compute(predictions=output, references=[[label] for label in label_batch]),
        "rouge": rouge.compute(predictions=output, references=label_batch),
        "meteor": meteor.compute(predictions=output, references=label_batch)
    }
    return metric_dict

In [15]:
model_inputs = title_dataset_test["inp"][:16]
right_answers = title_dataset_test["right_answer"][:16]
labels = [item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip() for item in title_dataset_test["distractors"][:16]]

In [16]:
predict_model_best = get_metric_inputs_seq2seq(model_inputs, model_best, tokenizer)
predict_model_last = get_metric_inputs_seq2seq(model_inputs, model_last, tokenizer)

In [17]:
labels

['Формы сложных слов. Как пользоваться смешающими словами. Водонепроницаемый Клот в лучшем.',
 'Отец Рождество опасен? Истинная история Святого Николая Традиции Рождества',
 'Истории о некоторых пациентах с гипертонией. Рассказ может помочь снизить кровяное давление. Предложения о том, как снизить кровяное давление.',
 'Как хорошо проводить время Благотворительные мероприятия во всем мире Выступления суперзвезд на благотворительных мероприятиях',
 'Неприятный друг Два невероятных случая Удар по стене',
 'Есть шансы, что они попытаются их схватить. Справляйтесь с несчастьем, будучи предопределённым. Лучший способ познать себя, выбрать бездомность',
 'Что означает три выстрела и два выстрела в лесу Как найти своих друзей, когда ты заблудился в лесу Самое важное — оставаться в одном месте',
 'Найти дом Жить с семьей Лучший приют',
 'Человек с Марса Что-то о коробках Футбольная игра',
 'Австралия, известная своим культурным разнообразием Австралия -- дом спорта Австралия -- фантастическая 

In [18]:
predict_model_best

['Как быть хорошим человеком Как быть хорошим человеком Как быть хорошим человеком',
 'Как Санта Клаус навещает своих детей Как Санта Клаус помогает бедным девушкам',
 'Как быть здоровым. Как быть здоровым. Как быть здоровым.',
 'Фестиваль Гластонбери в Великобритании Популярный фестиваль Гластонбери в Великобритании Популярный фестиваль',
 'Незабываемый опыт Незабываемый опыт Незабываемый опыт',
 'Как стать лучшим учеником Гарвардского университета Как стать лучшим студентом Гарвардского университета',
 'Как найти друзей Как найти друзей Как найти друзей',
 'Как жить с родителями Как жить с родителями Как жить с родителями',
 'Как жить на Марсе Как жить на Марсе Как жить на Марсе',
 'Австралийцы - лучшие спортсмены в мире Австралийцы -',
 'Как экономить энергию Постарайся быть энергичным. Как экономить энергию',
 'Развитие информационных технологий Развитие Интернета Развитие Интернета',
 'Мальчик и его мать Мальчик и его мать Мальчик и его мать',
 'Как есть. Как есть. Как есть.',
 'Н

In [19]:
predict_model_last

['Как хорошо одеваться Особая одежда в Англии Хорошая одежда и плохая одежда',
 'Отец Рождество спас троих девочек. Легенда Санта Клауса',
 'Как справиться с кровяным давлением. Как справляться с кровяным давлением. Как сохранять здоровье',
 'Рождественские фестивальы в Великобритании Большая помощь <unk> lt; <unk>lt; Гластон',
 'Удар по стене Два особых случая Удар по стене',
 'Лучший способ получить стипендию Гарвард -- город для молодых студентов Жизнь в колледже',
 'Путешествовать в лесу Что делать, если ты потеряешься? Как помочь другим',
 'Держаться подальше от мира Живи в убежищнице, ищи его. В',
 'Человек с Марса Жизнь на Марсе Интересная планета',
 'Климат в Австралии -- самый теплый и мягкий в мире Австралийцы -- мечта',
 'Измени свои привычки Экономьте энергию Печь с температурой и энергией',
 'Развитие информационной технологии История Интернета Информация в будущем',
 'В жаркое лето Страшный крокодил Опасность',
 'Ущерб, причиненный слишком большим количеством соли Здоровы

In [20]:
right_answers

['Связанные слова в повседневной жизни',
 'Легенда Санта-Клауса',
 'Лечение кровяного давления.',
 'Фестиваль Гластонбери',
 'Доброта незнакомцев',
 'От скамейки в парке до Гарвардского общежития Потрясающая история',
 'Что делать, если ты потеряешься в лесу',
 'Уход из дома',
 'Письмо с Марса',
 'Австралия -- самое живое место в мире',
 'Ежедневный энергетический цикл',
 'Образование в информационном возрасте',
 'Шрамы любви',
 'Смотри за солью.',
 'Радость Нового года по всему миру.',
 'Испытание любви']

In [25]:
compute_metrics(predict_model_best, labels)

{'bleu': {'bleu': 0.0,
  'precisions': [0.0872093023255814, 0.00641025641025641, 0.0, 0.0],
  'brevity_penalty': 0.7014180812626855,
  'length_ratio': 0.7381974248927039,
  'translation_length': 172,
  'reference_length': 233},
 'sbleu': {'score': 0.558697314056591,
  'counts': [15, 1, 0, 0],
  'totals': [172, 156, 140, 124],
  'precisions': [8.720930232558139,
   0.6410256410256411,
   0.35714285714285715,
   0.20161290322580644],
  'bp': 0.7014180812626855,
  'sys_len': 172,
  'ref_len': 233},
 'rouge': {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0},
 'meteor': {'meteor': 0.045494441084467666}}

In [26]:
compute_metrics(predict_model_last, labels)

{'bleu': {'bleu': 0.057582524419918166,
  'precisions': [0.21978021978021978,
   0.0963855421686747,
   0.05333333333333334,
   0.029850746268656716],
  'brevity_penalty': 0.7556176533910264,
  'length_ratio': 0.7811158798283262,
  'translation_length': 182,
  'reference_length': 233},
 'sbleu': {'score': 5.758252441991816,
  'counts': [40, 16, 8, 4],
  'totals': [182, 166, 150, 134],
  'precisions': [21.978021978021978,
   9.63855421686747,
   5.333333333333333,
   2.985074626865672],
  'bp': 0.7556176533910264,
  'sys_len': 182,
  'ref_len': 233},
 'rouge': {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0},
 'meteor': {'meteor': 0.176320067393745}}

In [47]:
BATCH_SIZE = 1
N_STEPS = (len(title_dataset_test) // BATCH_SIZE) + 1

metrics = []

for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):
    slice = title_dataset_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    if slice["inp"]:
        output_best = get_metric_inputs_seq2seq(slice["inp"], model_best, tokenizer)
        output_last = get_metric_inputs_seq2seq(slice["inp"], model_last, tokenizer)

        distractors = [item.replace('\n', '').replace('  ',' ').replace('  ',' ').strip() for item in slice["distractors"]]

        metrics_best = compute_metrics(output_best, distractors)
        metrics_last = compute_metrics(output_last, distractors)

        # код далее подходит только для батчей из одиночных примеров (BATCH_SIZE=1):
        metrics.append({
            "article": slice["article_ru"][0],
            "right_answer": slice["right_answer"][0],
            "distractors": distractors[0],
            "output_best": output_best[0],
            "output_last": output_last[0],

            "bleu_best": metrics_best["bleu"]["bleu"],
            "sbleu_best": metrics_best["sbleu"]["score"],
            "rouge1_best": metrics_best["rouge"]["rouge1"],
            "rouge2_best": metrics_best["rouge"]["rouge2"],
            "rougeL_best": metrics_best["rouge"]["rougeL"],
            "rougeLsum_best": metrics_best["rouge"]["rougeLsum"],
            "meteor_best": metrics_best["meteor"]["meteor"],

            "bleu_last": metrics_last["bleu"]["bleu"],
            "sbleu_last": metrics_last["sbleu"]["score"],
            "rouge1_last": metrics_last["rouge"]["rouge1"],
            "rouge2_last": metrics_last["rouge"]["rouge2"],
            "rougeL_last": metrics_last["rouge"]["rougeL"],
            "rougeLsum_last": metrics_last["rouge"]["rougeLsum"],
            "meteor_last": metrics_last["meteor"]["meteor"]
        })

/tmp/ipykernel_7813/2437664213.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N_STEPS), total=N_STEPS):


  0%|          | 0/243 [00:00<?, ?it/s]

In [48]:
metrics[0]

{'article': 'Девушка с голубыми глазами - голубоглазый. Мужчина с длинными ногами - это человек с длинными ногами. Женщина с белыми волосами - женщина с белыми волосами. Дети, у которых хорошая внешность, красивые дети. Что ты делаешь, когда хочешь купить одежду? Ты идешь в магазин. Если ты найдёшь одежду подходящего размера для тебя, и если она будет готова, ты, вероятно, купишь ее. Их называют готовой одеждой. Если вы не сможете найти одежду того размера, вы пойдёте в портной. Портной - это человек, который делает одежду. Он измерит вас тщательно, а потом сделает для вас одеяние. Такая одежда называется ручной одеждой.\nКак мы называем человека, который плохо одет? Мы называем его плохо одетым человеком. Женщина, хорошо одетая, называется хорошо одетая женщина.\nЧто ты носишь, когда идет сильный дождь? Ты носишь пальто, которое удержит дождь. Такое пальто называется плащом. Она сделана из водонепроницаемой ткани - зуб, который не позволяет воде пройти. В Англии много дождя. Если прие

In [52]:
metrics = pd.DataFrame(metrics)

In [53]:
metrics.describe()

,bleu_best,sbleu_best,rouge1_best,rouge2_best,rougeL_best,rougeLsum_best,meteor_best,bleu_last,sbleu_last,rouge1_last,rouge2_last,rougeL_last,rougeLsum_last,meteor_last
count,242.000000,242.000000,242.000000,242.0,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,0.004497,3.117486,0.004821,0.0,0.004821,0.004821,0.080099,0.048083,7.797526,0.013423,0.001377,0.013423,0.013423,0.148944
std,0.033097,4.202345,0.053462,0.0,0.053462,0.053462,0.100720,0.167598,16.228023,0.093211,0.021427,0.093211,0.093211,0.209340
min,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.236216,0.000000,0.000000,0.000000,0.000000,0.030675
50%,0.000000,2.449032,0.000000,0.0,0.000000,0.000000,0.049020,0.000000,3.089055,0.000000,0.000000,0.000000,0.000000,0.072464
75%,0.000000,4.047310,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,5.714697,0.000000,0.000000,0.000000,0.000000,0.175198
max,0.331808,33.180774,0.666667,0.0,0.666667,0.666667,0.531915,1.000000,100.000000,1.000000,0.333333,1.000000,1.000000,0.999852


In [54]:
metrics.to_excel("T5Metrics.xlsx", engine="openpyxl")

In [56]:
len(title_dataset_train), len(title_dataset_val), len(title_dataset_test)

(4375, 219, 242)

In [62]:
articles_train = title_dataset_train["article_ru"]
articles_test = title_dataset_test["article_ru"]
articles_val = title_dataset_val["article_ru"]

In [63]:
for idx, item in enumerate(articles_test):
    if item in articles_train:
        print(idx)

In [64]:
for idx, item in enumerate(articles_test):
    if item in articles_val:
        print(idx)

In [65]:
for idx, item in enumerate(articles_val):
    if item in articles_train:
        print(idx)